# Step 4: IRFs and DL3 scripts

In this notebook, we will run magic-cta-pipe scripts on a small data sample. Due to time constraints, we are unable to run the pipeline on all the files needed to produce the main plots (e.g. SED, light curves), so we will provide you a complete dataset to get to 'nice' plots and a few MCs and data *.h5* files to run the pipeline.
You have to provide the MCP scripts path and paths (+filenames) where you want to save logging infos.

In [32]:
import glob
import logging
import os
import subprocess
import sys
log = logging.getLogger()

### Instrument response function  

Here we will evaluate gammaness and $\theta$ cuts and create point-like IRFs, i.e. one angular bin every IRF (ring-wobble gammas), effective area and energy dispersion evaluation.

We can run the script directly in the terminal with (or here in the jupyter, as we show later):

$ python lst1_magic_create_irf.py -g yourfile -o Path1 -c configfile

with the followin options:

-g: input DL2 gamma file (test sample)

-o: output directory (to save IRFs)

-c: configuration file:

In the configuration file, we have to set the quality parameters as shown below:

1) IRF **quality_cuts**: "disp_diff_mean < 0.22", which represent the standard quality cut on DISPs.
2) **event_type**: "software", where the options are "hardware" which stands for a hardware trigger between MAGIC and LST-1; "software", which stands for any 2-telescope combinations except MAGIC-I + MAGIC-II; "software_only_3tel", which stands only for combos of the type MI + MII +LST-1; and "magic_only", which takes into account only MI + MII events.
3) **cut_type**: "dynamic", where the options are "global", which stands for a cut that is not energy dependent (i.e. a constant value); "dynamic", which stands for a cut evaluated in every energy bin according to the given efficiency.

In [33]:
%%capture
os.system("sed -n '199,244p' /home/raniere/Documentos/MAGIC/School_notebooks/config_dyn.yaml")

create_irf:
    quality_cuts: "disp_diff_mean < 0.22"
    event_type: "software"  # select "software", "software_only_3tel", "magic_only" or "hardware"
    weight_type_dl2: "intensity"  # select "simple", "variance" or "intensity"
    obs_time_irf: "50 h"  # used when creating a background HDU

    energy_bins:  # log space
        start: "0.01 TeV"
        stop: "1000 TeV"
        n_edges: 26

    migration_bins:  # log space
        start: 0.2
        stop: 5
        n_edges: 31

    fov_offset_bins:  # linear space, used for diffuse MCs
        start: "0 deg"
        stop: "1 deg"
        n_edges: 2

    source_offset_bins:  # linear space, used when creating PSF HDU
        start: "0 deg"
        stop: "1 deg"
        n_edges: 101

    bkg_fov_offset_bins:  # linear space, used when creating background HDU
        start: "0 deg"
        stop: "10 deg"
        n_edges: 21

    gammaness:
        cut_type: "dynamic"  # select "global" or "dynamic"
        global_cut_value: 0.8  # use

Let's start by setting up the configuration and data paths:

In [35]:
f=open('/home/raniere/Documentos/MAGIC/School_notebooks/IRF.log','w')

scripts='/home/raniere/Documentos/magic-cta-pipe/magicctapipe/scripts/lst1_magic/'
input_gamma='/home/raniere/Documentos/MAGIC/School_notebooks/DL2/*.h5'
output_dir_irf='/home/raniere/Documentos/MAGIC/School_notebooks/IRF/'
config='/home/raniere/Documentos/MAGIC/School_notebooks/config_dyn.yaml'

cd to the scripts directory to launch them

In [36]:
cd $scripts

/home/raniere/Documentos/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Lines to get files from the data directory

In [37]:
input_file_gamma = glob.glob(input_gamma)
input_file_gamma.sort()

Here we use python subprocess.run to run the script and also get a log file

In [38]:
for input_file in input_file_gamma: 
    x=subprocess.run(['python','lst1_magic_create_irf.py', f'-g{input_file}', f'-o{output_dir_irf}',\
        f'-c{config}'], stdout=f, stderr=f) 

We can check the log file by typing:

In [39]:
more /home/raniere/Documentos/MAGIC/School_notebooks/IRF.log

### Converting DL2 into DL3

Here we run a script that interpolates IRFs (to get e.g. gammaness cuts or effective area values for every run average pointing coordinates), produces DL3 data (i.e. DL2 data + gammaness and theta cuts) and packs them toghether with IRFs (*.fits.gz* files). 

Then we run the script below to create Header Data Unit (HDU) and observation index files (needed by Gammapy to process DL3 data and saved in the DL3 data directory).

We can run the script directly in the terminal with (or here in the jupyter, as we show later):

$ python lst1_magic_create_irf.py -d yourfile -i Path1 -o Path2 -c configfile

with the followin options:

-d: input file (DL2 real data)

-i: directory where you saved the IRFs

-o: directory to save DL3 data

-c: configuration file

In the configuration file, we set:

`interpolation_method: "linear"`, where the options are "nearest" which uses IRFs/cuts values evaluated in the nearest gamma test node; and "linear/cubic", which interpolates IRFs/cuts values by using more than one gamma test node.

In [40]:
%%capture
os.system("sed -n '245,255p' /home/raniere/Documentos/MAGIC/School_notebooks/config_dyn.yaml")

dl2_to_dl3:
    interpolation_method: "linear"  # select "nearest", "linear" or "cubic"
    source_name: "Crab"
    source_ra: null  #"83.633083 deg"  used when the source name cannot be resolved
    source_dec: null  #"22.0145 deg" used when the source name cannot be resolved


Let's settup the configuration and data paths:

In [41]:
f=open('/home/raniere/Documentos/MAGIC/School_notebooks/DL3.log','w')

input_data=('/home/raniere/Documentos/MAGIC/School_notebooks/data/*.h5')
input_dir_irf=('/home/raniere/Documentos/MAGIC/School_notebooks/IRF/')
output_dir_dl3=('/home/raniere/Documentos/MAGIC/School_notebooks/DL3/')
config=('/home/raniere/Documentos/MAGIC/School_notebooks/config_dyn.yaml')

cd to the scripts directory to launch them

In [42]:
cd $scripts

/home/raniere/Documentos/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Lines to get files from gamma/proton folders 

In [43]:
input_file_data = glob.glob(input_data)
input_file_data.sort()

Here we use python subprocess.run to run the scripts and get a log file

1) DL3 data 

In [44]:
for input_file in input_file_data:  
    y=subprocess.run(['python','lst1_magic_dl2_to_dl3.py', f'-d{input_file}', f'-i{input_dir_irf},\
        'f'-o{output_dir_dl3}', f'-c{config}'], stdout=f, stderr=f)        

2) Index files

In [45]:
z=subprocess.run(['python','lst1_magic_create_irf.py', f'-i{output_dir_dl3}'],stdout=f, stderr=f)        

In [46]:
more /home/raniere/Documentos/MAGIC/School_notebooks/DL3.log